In [2]:
!pip install web3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.5 MB/s eta 0:00:0000:0100:01
  Using cached eth_account-0.12.3-py3-none-any.whl (355 kB)
  Using cached pyunormalize-15.1.0-py3-none-any.whl
  Using cached eth_abi-5.1.0-py3-none-any.whl (29 kB)
  Using cached eth_hash-0.7.0-py3-none-any.whl (8.7 kB)
  Using cached eth_utils-4.1.1-py3-none-any.whl (96 kB)
  Using cached lru-dict-1.2.0.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Using cached hexbytes-0.3.1-py3-none-any.whl (5.9 kB)
  Using cached parsimonious-0.10.0-py3-none-any.whl (48 kB)
  Using cached bitarray-2.9.2-cp310-cp310-macosx_11_0_arm64.whl (124 kB)
  Using cached eth_keys-0.5.1-py3-none-any.whl (21 kB)
  Using cached ckzg-1.0.2-cp310-cp310-macosx_11_0_arm64.whl (85 kB)
  Using cached eth_account-0.12.2-py3-none-any.whl (355 kB)
  Using cached eth_account-0.12.1-py3-none-any.whl (355 kB)
  Using cached eth_account-0.12.0-py3-none-any.whl (354 kB)
  Using cached eth_account-0.11.2-py3-none-any.whl (355

In [24]:
import json
import math
from datetime import datetime, timedelta
import requests
from web3 import Web3

# Etherscan API endpoint and your API key
ETHERSCAN_API_URL = "https://api.etherscan.io/api"
ETHERSCAN_API_KEY = "BVRVIYIJ4H8EKBYVF5PAUFKR1DYK6BMN2D"  # Replace with your Etherscan API key


# Connect to Ethereum network
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/2a805701dd9f43479ebe914376e4ddc8'))

# Proxy contract address
PROXY_ADDRESS = '0x47176B2Af9885dC6C4575d4eFd63895f7Aaa4790'

# Implementation contract address
IMPLEMENTATION_ADDRESS = '0xaddd7bbedd9dd2e72649217ea784069821f44ab9'

PRECISION = 1e18
POOL_ID = 1

# Function to convert timestamp to date
def timestamp_to_date(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S UTC')

def get_contract_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": ETHERSCAN_API_KEY
    }
    response = requests.get(ETHERSCAN_API_URL, params=params)
    response_json = response.json()
    if response_json["status"] == "1":
        return json.loads(response_json["result"])
    else:
        raise Exception(f"Error fetching ABI: {response_json['result']}")

# Fetch ABI from Etherscan
try:
    ABI = get_contract_abi(IMPLEMENTATION_ADDRESS)
    print("Successfully fetched ABI from Etherscan")
except Exception as e:
    print(f"Failed to fetch ABI from Etherscan: {e}")
    print("Exiting...")
    exit(1)

# Initialize contract
contract = w3.eth.contract(address=PROXY_ADDRESS, abi=ABI)

def get_pool_data():
    try:
        pool = contract.functions.pools(POOL_ID).call()
        pool_data = contract.functions.poolsData(POOL_ID).call()
        total_deposited = contract.functions.totalDepositedInPublicPools().call()
        
        pool_info = {
            'payoutStart': pool[0],
            'decreaseInterval': pool[1],
            'withdrawLockPeriod': pool[2],
            'claimLockPeriod': pool[3],
            'withdrawLockPeriodAfterStake': pool[4],
            'initialReward': pool[5],
            'rewardDecrease': pool[6],
            'minimalStake': pool[7],
            'isPublic': pool[8]
        }
        
        pool_data_info = {
            'lastUpdate': pool_data[0],
            'rate': pool_data[1],
            'totalVirtualDeposited': pool_data[2]
        }
        
        return pool_info, pool_data_info, total_deposited
    except Exception as e:
        print(f"Error fetching pool data: {e}")
        return None, None, None

def simulate_stake_multipliers(address):
    pool_info, pool_data_info, total_deposited = get_pool_data()
    
    print("\n--- Pool Information ---")
    print(f"Payout Start: {timestamp_to_date(pool_info['payoutStart'])}")
    print(f"Decrease Interval: {pool_info['decreaseInterval']} seconds")
    print(f"Withdraw Lock Period: {pool_info['withdrawLockPeriod']} seconds")
    print(f"Claim Lock Period: {pool_info['claimLockPeriod']} seconds")
    print(f"Withdraw Lock Period After Stake: {pool_info['withdrawLockPeriodAfterStake']} seconds")
    print(f"Initial Reward: {pool_info['initialReward']} (raw value)")
    print(f"Reward Decrease: {pool_info['rewardDecrease']} (raw value)")
    print(f"Minimal Stake: {pool_info['minimalStake']} (raw value)")
    print(f"Is Public: {pool_info['isPublic']}")
    
    print("\n--- Pool Data ---")
    print(f"Last Update: {timestamp_to_date(pool_data_info['lastUpdate'])}")
    print(f"Current Rate: {pool_data_info['rate']} (raw value)")
    print(f"Total Virtual Deposited: {pool_data_info['totalVirtualDeposited']} (raw value)")
    
    print(f"\nTotal Deposited in Public Pools: {total_deposited} (raw value)")
    
    current_time = w3.eth.get_block('latest')['timestamp']
    print(f"\nCurrent Time: {timestamp_to_date(current_time)}")
    
    stake_durations = [0, 6*30*24*3600, 9*30*24*3600, 12*30*24*3600, 18*30*24*3600]  # 0, 6, 9, 12, 18 months in seconds
    
    print("\n--- Simulated Stake Multipliers ---")
    for duration in stake_durations:
        end_time = current_time + duration
        multiplier = contract.functions.getClaimLockPeriodMultiplier(POOL_ID, current_time, end_time).call()
        months = duration / (30*24*3600)
        print(f"Stake Duration: {months:.1f} months, Multiplier: {multiplier / PRECISION:.4f}")

# Address to analyze (not used in this simulation, but kept for future reference)
address = '0xA946525128aFDB3854e89943a21757BB73A2048C'

simulate_stake_multipliers(address)

Successfully fetched ABI from Etherscan

--- Pool Information ---
Payout Start: 2024-02-08 12:00:00 UTC
Decrease Interval: 86400 seconds
Withdraw Lock Period: 0 seconds
Claim Lock Period: 7776000 seconds
Withdraw Lock Period After Stake: 0 seconds
Initial Reward: 3456000000000000000000 (raw value)
Reward Decrease: 592558728240000000 (raw value)
Minimal Stake: 0 (raw value)
Is Public: False

--- Pool Data ---
Last Update: 2024-08-13 10:52:59 UTC
Current Rate: 99915932702180595705140893711715164402275172 (raw value)
Total Virtual Deposited: 106965673 (raw value)

Total Deposited in Public Pools: 57224396229344089483745 (raw value)

Current Time: 2024-08-13 11:56:11 UTC

--- Simulated Stake Multipliers ---
Stake Duration: 0.0 months, Multiplier: 10000000.0000
Stake Duration: 6.0 months, Multiplier: 10541981.2656
Stake Duration: 9.0 months, Multiplier: 15783106.0897
Stake Duration: 12.0 months, Multiplier: 20990462.8783
Stake Duration: 18.0 months, Multiplier: 31263904.0836
